In [31]:
# Notebook to study the gammas emmited from the 2nubb events
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import os
import sys
import tables as tb
import numpy  as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from matplotlib.colors import LogNorm  # Import LogNorm for logarithmic color scale
from matplotlib.cm import get_cmap

# Create a sequential colormap
cmap = get_cmap('Spectral')  # You can choose any sequential colormap here

In [32]:
# Loading in the file
files = os.listdir("../data/nexus/1bar/")

# Get the single elctron efiles
files = [filename for filename in files if 'single' in filename]
files = sorted(files)

files.append("xesphere_bb_1_bar_2.5_MeV.h5")

print(files)

config_v = []
parts_v  = []
hits_v   = []
energy_v = []
pressure_v = []
 
 
for index, f in enumerate(files):
    config_v.append(pd.read_hdf("../data/nexus/1bar/" + f, 'config'))
    parts_v .append(pd.read_hdf("../data/nexus/1bar/" + f, 'parts'))
    hits_v  .append(pd.read_hdf("../data/nexus/1bar/" + f, 'hits'))

    eventtype = config_v[index][config_v[index]["param_key"] == "/nexus/RegisterGenerator"].iloc[0]['param_value']

    if (eventtype == "Decay0Interface"):
        eventtype = "bb"
        energy = "0nuBB"
        energy_v.append(energy)
    else:
        eventtype = "singleeminus"

        # Then also get the energy
        energy = config_v[index][ config_v[index]["param_key"] == "/Generator/SingleParticle/min_energy"].iloc[0]['param_value']
        energy = [float(num) if '.' in num else int(num) for num in energy.split() if num.replace('.', '', 1).isdigit()]
        energy_v.append(energy[0])
        print(energy[0])

    pressure = config_v[index][ config_v[index]["param_key"] == "/Geometry/XeSphere/pressure"].iloc[0]['param_value']
    pressure = [float(num) if '.' in num else int(num) for num in pressure.split() if num.replace('.', '', 1).isdigit()]
    pressure = f"{int(pressure[0])}_bar"
    pressure_v.append(pressure)

display(config_v[0])


['xesphere_singleeminus_1_bar_0.1_MeV.h5', 'xesphere_singleeminus_1_bar_0.25_MeV.h5', 'xesphere_singleeminus_1_bar_0.5_MeV.h5', 'xesphere_singleeminus_1_bar_1_MeV.h5', 'xesphere_singleeminus_1_bar_2.5_MeV.h5', 'xesphere_bb_1_bar_2.5_MeV.h5']
0.1
0.25
0.5
1
2.5


,param_key,param_value
0,event_type,background
1,num_events,5000
2,saved_events,5000
3,interacting_events,5000
4,/PhysicsList/RegisterPhysics,G4EmStandardPhysics_option4
5,/PhysicsList/RegisterPhysics,G4DecayPhysics
6,/PhysicsList/RegisterPhysics,G4RadioactiveDecayPhysics
7,/PhysicsList/RegisterPhysics,NexusPhysics
8,/PhysicsList/RegisterPhysics,G4StepLimiterPhysics
9,/nexus/RegisterGeometry,XeSphere


In [33]:
# Lets study one sample
pd.set_option('display.max_rows', None)
event = parts_v[4][ parts_v[4]["event_id"] == 3164 ]
# display(event)
display(event[event["particle_name"] == "gamma"])
print(event.creator_proc.unique())

print(np.sqrt(event[event["particle_name"] == "gamma"]['final_x']*event[event["particle_name"] == "gamma"]['final_x'] + event[event["particle_name"] == "gamma"]['final_y']*event[event["particle_name"] == "gamma"]['final_y'] + event[event["particle_name"] == "gamma"]['final_z']*event[event["particle_name"] == "gamma"]['final_z'])/1000)

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,kin_energy,length,creator_proc,final_proc
2065472,3164,363,gamma,0,56,-25.760513,547.165710,104.839264,4.804668,-30.285427,544.890564,101.098907,4.825669,0.004097,6.296134,eIoni,phot
2065499,3164,53,gamma,0,1,-57.786278,523.212158,83.699753,4.341448,-57.604652,523.255981,83.715469,4.342073,0.001534,0.187500,eBrem,phot
2065530,3164,47,gamma,0,1,-185.169250,556.335022,163.958618,3.740025,17182.503906,-7535.739258,3810.456299,86.232094,0.644715,24730.500000,eBrem,phot
2065901,3164,750,gamma,0,14,36.585667,51.454411,186.293869,0.717466,37.012356,51.339798,186.404602,0.718985,0.003982,0.455476,eIoni,phot


['none' 'eIoni' 'phot' 'eBrem' 'compt']
2065472     0.555017
2065499     0.533032
2065530    19.145374
2065901     0.196856
dtype: float32


In [34]:
# Hit vector 
event_hits = hits_v[4][ hits_v[4]["event_id"] == 3164 ]
display(event_hits[ event_hits["particle_id"] == 52])

,event_id,x,y,z,time,energy,particle_id,hit_id
46666786,3164,-73.505219,521.297852,85.779121,4.284910,0.000020,52,0
46666787,3164,-73.504219,521.302979,85.783218,4.285236,0.000075,52,1
46666788,3164,-73.509430,521.303345,85.779953,4.285549,0.000026,52,2
46666789,3164,-73.513771,521.306763,85.777672,4.285857,0.000205,52,3
46666790,3164,-73.513954,521.307312,85.772964,4.286127,0.000312,52,4
46666791,3164,-73.514084,521.308716,85.770325,4.286342,0.000385,52,5
46666792,3164,-73.514198,521.309692,85.771194,4.286508,0.000179,52,6


In [35]:
# get the hits of a specific event
x = event_hits.x/1000
y = event_hits.y/1000
z = event_hits.z/1000
e = event_hits.energy*1000

In [36]:
# Lets plot this event
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')

x_range = (x.max()-x.min()) * 0.5
y_range = (y.max()-y.min()) * 0.5
z_range = (z.max()-z.min()) * 0.5
mid_x = (x.max()+x.min()) * 0.5
mid_y = (y.max()+y.min()) * 0.5
mid_z = (z.max()+z.min()) * 0.5
ax.set_xlim(mid_x - x_range, mid_x + x_range)
ax.set_ylim(mid_y - y_range, mid_y + y_range)
ax.set_zlim(mid_z - z_range, mid_z + z_range)

# ax.set_xlim(-0.2, 0.2)
# ax.set_ylim(-0.2, 0.2)
# ax.set_zlim(-0.2, 0.2)

p = ax.scatter(x,y,z,cmap='viridis',c=e) #coolwarm
cb = fig.colorbar(p, ax=ax)
cb.set_label('Energy (keV)')

plt.show()